# SQL Windows Functions
L'objectif de ce notebook est de vous introduire aux notions de windows function. Celui-ci n'a donc pas pour but d'être exhaustif mais de vous donner des clés de compréhensions pour ce sujet. 


## Qu'est ce qu'une window function ? 
Une window function est fonction SQL ou le résultat est tiré d'un d'une "fenêtre" (un set d'une ou plusieurs lignes). En gros, ces fonctions nous donnent des outils pour filter et/ou ordonner un sous ensemble de données sur lesquels on peut appliquer des fonctions.

En plus de ces outils de "fenêtrage", nous avons aussi un ensemble de fonction bien spécifique (numéroter des lignes, sélectionner la première valeur d'un sous ensemble ordonner, sélectionner la valeur suivant d'un sous ensemble ordonner...) 

Nous verrons ici des uses cases et des exercices pour vous faire la main.

In [4]:
# configuration de l'environnement 
from google.cloud import bigquery
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = \
    "/Users/garantme/Downloads/datawarehouse-267911-7a35fd65f9dd.json"

%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


### Premier use case : les dates de premier paiement
Commençons doucement, nous souhaitons récupérer la date du premier paiement associé à une application

In [11]:
%%bigquery

# Requête pour récupérer tous les paiements de chaque application
SELECT 
    AI.applicationid, 
    I.paymentsucceededat
FROM `datawarehouse-267911.db_production_console.garantme_ApplicationInvoice` AI
INNER JOIN `datawarehouse-267911.db_production_console.garantme_PaymentPlan` PP ON AI.id=PP.applicationinvoiceid 
INNER JOIN `datawarehouse-267911.db_production_console.garantme_Installment` I ON PP.id=I.paymentplanid
INNER JOIN `datawarehouse-267911.db_production_console.garantme_InstallmentPaymentStatus` IPS ON I.installmentpaymentstatusid=IPS.id
WHERE 
    AI.deletedat IS NULL
    AND PP.deletedat IS NULL 
    AND I.deletedat IS NULL
    AND IPS.name IN('Succeeded')
LIMIT 10

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  9.65rows/s]


,applicationid,paymentsucceededat
0,230201,2021-07-11 18:37:36.157000+00:00
1,211402,2021-07-11 19:03:56.437000+00:00
2,219491,2021-07-12 00:00:41.605000+00:00
3,111104,2021-07-11 23:30:39.735000+00:00
4,218430,2021-07-11 18:04:53.953000+00:00
5,229901,2021-07-16 12:09:01.885000+00:00
6,236710,2021-07-16 12:15:22.001000+00:00
7,237838,2021-07-16 14:53:44.560000+00:00
8,230017,2021-07-16 13:05:34.001000+00:00
9,231119,2021-07-16 13:06:34.220000+00:00


In [12]:
%%bigquery

# Requête pour récupérer les premiers paiement
SELECT 
    AI.applicationid, 
    FIRST_VALUE(I.paymentsucceededat) OVER (
        PARTITION BY AI.applicationid
        ORDER BY I.paymentsucceededat
    ) AS premier_paiement
FROM `datawarehouse-267911.db_production_console.garantme_ApplicationInvoice` AI
INNER JOIN `datawarehouse-267911.db_production_console.garantme_PaymentPlan` PP ON AI.id=PP.applicationinvoiceid 
INNER JOIN `datawarehouse-267911.db_production_console.garantme_Installment` I ON PP.id=I.paymentplanid
INNER JOIN `datawarehouse-267911.db_production_console.garantme_InstallmentPaymentStatus` IPS ON I.installmentpaymentstatusid=IPS.id
WHERE 
    AI.deletedat IS NULL
    AND PP.deletedat IS NULL 
    AND I.deletedat IS NULL
    AND IPS.name IN('Succeeded')
LIMIT 10

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11.94rows/s]


,applicationid,premier_paiement
0,158,2018-02-05 21:44:14+00:00
1,176,2018-02-02 00:00:00+00:00
2,176,2018-02-02 00:00:00+00:00
3,198,2018-03-28 20:43:46+00:00
4,278,2018-04-10 10:13:41+00:00
5,284,2018-04-19 20:38:49+00:00
6,284,2018-04-19 20:38:49+00:00
7,284,2018-04-19 20:38:49+00:00
8,584,2018-04-06 12:30:32+00:00
9,641,2018-07-26 10:54:43+00:00


Regardons ce que nous venons de faire.
La fonction "OVER" permet de créer notre fameuse fenêtre (le sous ensemble sur lequel nous voulons obtenir un résultat). Ce OVER possède trois arguments : 
 - PARTITION BY
 - ORDER BY
 - ROWS BETWEEN
 
Mettons le rows between de côté pour le moment. Dans l'exemple ci-dessus, nous avons spécifié PARTITION BY (qui revient à une clause GROUP BY) et un ORDER BY. On créer donc une partition par application. Cette partition est trié par date de paiement. Pour cette partition triée, on sélectionne la première valeur avec "FIRST_VALUE"

Pour le plus sagaces d'entre vous, vous vous dites qu'on aurait pu faire ça avec un GROUP BY et un MIN. Oui. Les choses se compliqueront par la suite. Notons toutefois une première différence avec un GROUP BY, la windows function renvoie une résultat par input, alors que le GROUP BY aggrège.

Compliquons tout de suite les choses pour montrer la vraie plus value de ce genre de méthode : nous souhaitons numéroter les paiements en fonction de leur date de survenance

In [15]:
%%bigquery

# Requête pour récupérer les premiers paiement
SELECT 
    AI.applicationid, 
    ROW_NUMBER() OVER (
        PARTITION BY AI.applicationid
        ORDER BY I.paymentsucceededat
    ) AS numero_paiement
FROM `datawarehouse-267911.db_production_console.garantme_ApplicationInvoice` AI
INNER JOIN `datawarehouse-267911.db_production_console.garantme_PaymentPlan` PP ON AI.id=PP.applicationinvoiceid 
INNER JOIN `datawarehouse-267911.db_production_console.garantme_Installment` I ON PP.id=I.paymentplanid
INNER JOIN `datawarehouse-267911.db_production_console.garantme_InstallmentPaymentStatus` IPS ON I.installmentpaymentstatusid=IPS.id
WHERE 
    AI.deletedat IS NULL
    AND PP.deletedat IS NULL 
    AND I.deletedat IS NULL
    AND IPS.name IN('Succeeded')
LIMIT 10

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11.55rows/s]


,applicationid,numero_paiement
0,538,1
1,538,2
2,1191,1
3,1350,1
4,3323,1
5,3796,1
6,4845,1
7,6167,1
8,6167,2
9,6167,3


### Deuxième use case
Lag and Lead

### Troisième use case 
